In [1]:
%run ../../import_envs.py
%run ../apg_hyperparams.py

CUDA = torch.cuda.is_available()
DEVICE = torch.device('cuda:1')

probtorch: 0.0+5a2c637 torch: 1.3.0
inference method:apg, apg sweeps:1, epochs:500, sample size:10, batch size:10, learning rate:0.0001


In [2]:
# train data path
DATA_PATHS = []
TRAIN_DATA_DIR = '/data/hao/apg_data/bmnist/train/'
print('===Loading training data from %s===' % TRAIN_DATA_DIR)
for file in os.listdir(TRAIN_DATA_DIR):
    DATA_PATHS.append(os.path.join(TRAIN_DATA_DIR, file))
print('===%d groups of data files are loaded.===' % len(DATA_PATHS))

===Loading training data from /data/hao/apg_data/bmnist/train/===
===60 groups of data files are loaded.===


In [3]:
from apg_modeling import init_model
from affine_transformer import Affine_Transformer

model, optimizer = init_model(frame_pixels=FRAME_PIXELS,
                               digit_pixels=DIGIT_PIXELS, 
                               num_hidden_digit=NUM_HIDDEN_DIGIT, 
                               num_hidden_coor=NUM_HIDDEN_COOR, 
                               z_where_dim=Z_WHERE_DIM, 
                               z_what_dim=Z_WHAT_DIM, 
                               CUDA=CUDA, 
                               DEVICE=DEVICE, 
                               LOAD_VERSION=None, 
                               LR=LR)
AT = Affine_Transformer(frame_pixels=FRAME_PIXELS, 
                        digit_pixels=DIGIT_PIXELS, 
                        CUDA=CUDA, 
                        DEVICE=DEVICE)

In [4]:
%run ../apg_training.py

train(optimizer=optimizer, 
      model=model,
      AT=AT,
      apg_sweeps=APG_SWEEPS,
      data_paths=DATA_PATHS,
      mnist_mean_path=MNIST_MEAN_PATH, 
      K=K,
      num_epochs=NUM_EPOCHS,
      sample_size=SAMPLE_SIZE,
      batch_size=BATCH_SIZE,
      CUDA=CUDA,
      DEVICE=DEVICE,
      MODEL_VERSION=MODEL_VERSION)

Start to train the APG samplers with 60 groups of datasets...


/home/hao/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:2751: UserWarning: Default grid_sample and affine_grid behavior will be changed to align_corners=False from 1.4.0. See the documentation of grid_sample for details.
  warnings.warn("Default grid_sample and affine_grid behavior will be changed "
/home/hao/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:2693: UserWarning: Default grid_sample and affine_grid behavior will be changed to align_corners=False from 1.4.0. See the documentation of grid_sample for details.
  warnings.warn("Default grid_sample and affine_grid behavior will be changed "


Epoch=0, Group=0 completed in (56s),  
Epoch=0, Group=1 completed in (56s),  
Epoch=0, Group=2 completed in (56s),  
Epoch=0, Group=3 completed in (56s),  
Epoch=0, Group=4 completed in (56s),  
Epoch=0, Group=5 completed in (56s),  
Epoch=0, Group=6 completed in (56s),  


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/hao/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3325, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-4-7f5dcd85ad28>", line 15, in <module>
    MODEL_VERSION=MODEL_VERSION)
  File "/home/hao/Research/amortized/BMNIST/apg_training.py", line 52, in train
    density_required=density_required)
  File "/home/hao/Research/amortized/BMNIST/apg_objective.py", line 66, in apg_objective
    density_required=density_required)
  File "/home/hao/Research/amortized/BMNIST/apg_objective.py", line 167, in rws
    z_where_old_t_1=None)
  File "/home/hao/Research/amortized/BMNIST/apg_objective.py", line 127, in propose_one_movement
    q_k_f = enc_coor.forward(conved=conved_k, sampled=True)
  File "../modules/enc_coor.py", line 31, in forward
    q.normal(loc=q_mean, scale=q_std, value=z_where, name='z_where')
  File "<string>", line 2, in f
  File "/home/hao/Research/probtorch/p

ValueError: NaN log prob encountered in nodewith name: z_where

In [ ]:
%debug

> /home/hao/Research/probtorch/probtorch/stochastic.py(187)__setitem__()
    185         if (node.log_prob != node.log_prob).sum() > 0:
    186             raise ValueError("NaN log prob encountered in node"
--> 187                              "with name: " + name)
    188         self._nodes[name] = node
    189 



ipdb>  u


> /home/hao/Research/probtorch/probtorch/stochastic.py(292)variable()
    290             self.append(node)
    291         else:
--> 292             self[name] = node
    293         return value
    294 



ipdb>  u


> <string>(2)f()



...

*** NameError: name 'conved_k' is not defined


ipdb>  u


> /home/hao/Research/amortized/BMNIST/apg_objective.py(127)propose_one_movement()
    125         CP = conved_k.shape[-1] # convolved output pixels ##  S * B * CP * CP
    126         conved_k = F.softmax(conved_k.squeeze(0).view(S, B, CP, CP).view(S, B, CP*CP), -1) ## S * B * 1639
--> 127         q_k_f = enc_coor.forward(conved=conved_k, sampled=True)
    128         z_where_k = q_k_f['z_where'].value
    129         z_where.append(z_where_k.unsqueeze(2)) ## expand to S * B * 1 * 2



ipdb>  conved_k


tensor([[[1.3336e-15, 1.3336e-15, 1.3336e-15,  ..., 1.3336e-15,
          1.3336e-15, 1.3336e-15],
         [9.1190e-20, 9.1190e-20, 9.1190e-20,  ..., 9.1190e-20,
          9.1190e-20, 9.1190e-20],
         [2.8625e-22, 2.8625e-22, 2.8625e-22,  ..., 2.8625e-22,
          2.8625e-22, 2.8625e-22],
         ...,
         [4.6983e-19, 4.6983e-19, 4.6983e-19,  ..., 4.6983e-19,
          4.6983e-19, 4.6983e-19],
         [1.7095e-19, 1.7095e-19, 1.7095e-19,  ..., 1.7095e-19,
          1.7095e-19, 1.7095e-19],
         [1.0783e-06, 1.0683e-05, 9.0844e-05,  ..., 3.8147e-20,
          3.8147e-20, 3.8147e-20]],

        [[1.3336e-15, 1.3336e-15, 1.3336e-15,  ..., 1.3336e-15,
          1.3336e-15, 1.3336e-15],
         [9.1190e-20, 9.1190e-20, 9.1190e-20,  ..., 9.1190e-20,
          9.1190e-20, 9.1190e-20],
         [2.8625e-22, 2.8625e-22, 2.8625e-22,  ..., 2.8625e-22,
          2.8625e-22, 2.8625e-22],
         ...,
         [4.6983e-19, 4.6983e-19, 4.6983e-19,  ..., 4.6983e-19,
          4.698